In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")


import spartan.utils.utils as spartan_utils
from imitation_agent.dataset.function_factory import ObservationFunctionFactory, ActionFunctionFactory
from imitation_agent.dataset.imitation_episode_sequence_dataset import ImitationEpisodeSequenceDataset
from imitation_agent.dataset.imitation_episode_dataset import ImitationEpisodeDataset
from imitation_agent.training import train_utils

spartan_source_dir = spartan_utils.getSpartanSourceDir()
logs_config_yaml = os.path.join(spartan_source_dir, "modules/imitation_agent/config/task/move_to_box_se2_box_in_frame.yaml")
logs_config = spartan_utils.getDictFromYamlFilename(logs_config_yaml)

imitation_src_dir = os.path.join(spartan_source_dir, "modules/imitation_agent")
data_dir = spartan_utils.get_data_dir()
logs_dir_path = os.path.join(data_dir, "pdc/imitation/move_to_box_se2")

config_yaml = os.path.join(imitation_src_dir, "config", "model", "mlp_stateless_position.yaml")
config = spartan_utils.getDictFromYamlFilename(config_yaml)

NUM_LOGS = 30 # can leave this at 30, just need to build a dataset to grab a couple things like K matrices
logs_config_downsampled = train_utils.deterministic_downsample(logs_config, NUM_LOGS)
NUM_LOGS=str(NUM_LOGS)

obs_function = ObservationFunctionFactory.get_function(config)
action_function = ActionFunctionFactory.action_from_config(config)
dataset = ImitationEpisodeDataset(logs_dir_path, logs_config_downsampled, config,
                                  action_function=action_function,
                                  observation_function=obs_function)

In [ ]:
import os
from matplotlib  import cm
import matplotlib

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import spartan.utils.utils as spartan_utils

spartan_source_dir = spartan_utils.getSpartanSourceDir()

from spartan.utils import transformations
from imitation_agent.utils.visibility_utils import check_sugar_box_in_frame

def prune_out_of_FOV(df, dataset):
    # these don't move
    episode = dataset.episodes[dataset.episodes.keys()[0]]
    camera_num = 0
    T_W_camera = episode.get_camera_pose_matrix(camera_num)
    K = episode.get_K_matrix(camera_num)
    
    in_frame_list = []
    for index, row in df.iterrows():
        object_position = row["object_position"].strip("[").strip("]").split(",")
        object_position = [float(x) for x in object_position]
        object_position = np.asarray(object_position)
        
        object_rpy = row["object_rpy"].strip("[").strip("]").split(",")
        object_rpy = [float(x) for x in object_rpy]
            
        static_rotation_only = transformations.euler_matrix(0.0, 0.0, object_rpy[2], axes='sxyz')
        static_rotation_only[0,3] = object_position[0]
        static_rotation_only[1,3] = object_position[1]
        static_rotation_only[2,3] = 2.00476981e-02
        T_W_B = static_rotation_only
        in_frame = check_sugar_box_in_frame(K, T_W_camera=T_W_camera, T_W_B=T_W_B, urange=[0,640], vrange=[0,480])
        
        if not in_frame:
            in_frame_list.append(False)
        else:
            in_frame_list.append(True)
    
    df["in_frame"] = in_frame_list
    print len(df)
    print len(df[df["in_frame"] == True])
    return df[df["in_frame"] == True]

def adjust_reward(df):
    df['reward'] += 1.2
    df['reward'] = np.minimum(0.0,df['reward'])
    return df


def get_min_max_rewards(df):
    first = True
    for index, row in df.iterrows():
        if first:
            min_r = row["reward"]
            max_r = row["reward"]
            first = False
            continue
        if row["reward"] > max_r:
            max_r = row["reward"]
        if row["reward"] < min_r:
            min_r = row["reward"]
    return min_r, max_r
        
def get_normed_reward(reward, min_r, max_r):
    return (reward - min_r)/(max_r-min_r)
            
dfs = []  


folders = []
unique_names = []

for num_logs in [30,50,100,200]:
    NUM_LOGS = str(num_logs)
    for folder in sorted(os.listdir(os.path.join(spartan_source_dir, "sandbox/experiment_02/logs-"+NUM_LOGS))):
        if "bak" in folder or folder == "05-endtoend" or folder =="01-gt-pose":
            continue
        folders.append(folder)
        path_to_move_to_box_folder = os.path.join(spartan_source_dir, "sandbox","experiment_02/logs-"+NUM_LOGS,folder)
        path_to_csv = os.path.join(path_to_move_to_box_folder, "results.csv")
        df = pd.read_csv(path_to_csv, index_col=0)
        name = folder[3:]
        df["name"] = name
        print name
        if num_logs == 30:
            unique_names.append(name)
        df["num_logs"] = NUM_LOGS
        dfs.append(df)
        
    
for i, df in enumerate(dfs):
    dfs[i] = prune_out_of_FOV(df, dataset)
    
for i, df in enumerate(dfs):
    dfs[i] = adjust_reward(df)   

In [ ]:
import matplotlib

def create_violin_plot(dfs):
    df_all = pd.concat(dfs)
    sns.set(style="whitegrid")
    ax = sns.violinplot(x=df_all["name"], y=df_all["reward"])
    fig = matplotlib.pyplot.gcf()
    fig.set_size_inches(15, 4)
    plt.show()
    
create_violin_plot(dfs)

In [ ]:
def create_violin_plot_for_name(dfs, name):
    df_all = pd.concat(dfs)
    df_all.head()
    print name
    df_for = df_all[df_all['name'].str.match(name)]
    sns.set(style="whitegrid")
    ax = sns.violinplot(x=df_for["num_logs"], y=df_for["reward"],  order=["30", "50", "200"])
    ax.set(ylim=(-5,0))
    fig = matplotlib.pyplot.gcf()
    fig.set_size_inches(15, 4)
    plt.show()
    
for unique_name in unique_names:
    create_violin_plot_for_name(dfs, unique_name)

In [ ]:
def create_violin_plot_for_num(dfs, num):
    df_all = pd.concat(dfs)
    df_all.head()
    print num
    df_for = df_all[df_all['num_logs'].str.match(str(num))]
    
    sns.set(style="whitegrid")
    ax = sns.violinplot(x=df_for["name"], y=df_for["reward"])
    ax.set(ylim=(-5,0))
    fig = matplotlib.pyplot.gcf()
    fig.set_size_inches(15, 4)
    plt.show()
    
for num_logs in [30,50,100,200]:
    create_violin_plot_for_num(dfs, num_logs)

In [ ]:
def create_boxplot_plot_for_name(dfs, name):
    df_all = pd.concat(dfs)
    df_all.head()
    print name
    df_for = df_all[df_all['name'].str.match(name)]
    sns.set(style="whitegrid")
    ax = sns.boxplot(x=df_for["num_logs"], y=df_for["reward"],  order=["30", "50", "100", "200"])
    ax.set(ylim=(-10,1))
    fig = matplotlib.pyplot.gcf()
    fig.set_size_inches(15, 4)
    plt.show()
    
for unique_name in unique_names:
    create_boxplot_plot_for_name(dfs, unique_name)

In [ ]:
print unique_names

sns.set_style("white", {"axes.grid": True})

unique_names_to_percent_above = dict()
for unique_name in unique_names:
    unique_names_to_percent_above[unique_name] = dict()
    unique_names_to_percent_above[unique_name]["num_logs"] = []
    unique_names_to_percent_above[unique_name]["percent_within"] = []
    
def folder_to_plot_name(name):
    if name == "gt-3D-points":
        return "Ground truth 3D points"
    if name == "gt-3D-points-projected":
        return "Ground truth 2D image coordinates"
    if name == "endtoend-fullwidth":
        return "End-to-End"
    else:
        return "Dense Descriptors"


for df in dfs:
    num_within_threshold = len(df[df["reward"] >= 0.0])
    num_total = len(df)
    name = df.iloc[0]["name"]
    if name == "gt-pose":
        continue
    num_logs = df.iloc[0]["num_logs"]
    percent_within = num_within_threshold * 1.0/ num_total
    print name, num_logs, percent_within
    unique_names_to_percent_above[name]["num_logs"].append(num_logs)
    unique_names_to_percent_above[name]["percent_within"].append(percent_within)
    
colors = ["red", "orange", "blue", "green", "purple"]
methods = ['gt-3D-points', 'gt-3D-points-projected', 'endtoend-fullwidth', 'dd-d_images-SURF-16-10D']

for unique_name, color in zip(methods,colors):
    num_logs = unique_names_to_percent_above[unique_name]["num_logs"]
    percents_within = unique_names_to_percent_above[unique_name]["percent_within"]
    plt.plot(num_logs, percents_within, color=color, label=folder_to_plot_name(unique_name))
    
plt.legend(loc='upper right', bbox_to_anchor=(1.78,1.03), frameon=True)
plt.xlabel("# demonsrations")
plt.ylabel("percent success within threshold")
plt.savefig("MTBSE2-samplecomplexity-line.pdf", bbox_inches="tight")
plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def create_boxplot_for_num_on_axis(dfs, num, ax_on):
    df_all = pd.concat(dfs)
    df_all.head()
    print num
    df_for = df_all[df_all['num_logs'].str.match(str(num))]
    
    sns.set(style="whitegrid")
    sns.boxplot(x=df_for["name"], y=df_for["reward"], ax=ax_on)
    
f, axes = plt.subplots(1, 4, figsize=(7, 3), sharex=True)
    
for i, num_logs in enumerate([30,50,100,200]):
    create_boxplot_for_num_on_axis(dfs, num_logs, axes[i])
    axes[i].set(ylim=(-10,1))
    

fig = matplotlib.pyplot.gcf()
fig.set_size_inches(15, 4)
plt.show()

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def create_boxplot_plot_for_name_on_axes(dfs, name, ax_on, color):
    df_all = pd.concat(dfs)
    df_all.head()
    print name
    df_for = df_all[df_all['name'] == name]
    df_for = df_for[df_for['reward'] > -40]
    sns.set_style("white", {"axes.grid": True})
    sns.violinplot(x=df_for["num_logs"], y=df_for["reward"],  order=["30", "50", "100", "200"], ax=ax_on, cut=0, color=color)
    #sns.swarmplot(x=df_for["num_logs"], y=df_for["reward"],  order=["30", "50", "100", "200"], ax=ax_on)
        
f, axes = plt.subplots(1, 4, figsize=(7, 3), sharex=True)
    
i = 0
for unique_name, color in zip(methods,colors):
    create_boxplot_plot_for_name_on_axes(dfs, unique_name, axes[i], color)
    axes[i].set(ylim=(-10,1))
    if i > 0:
        axes[i].set_ylabel("")
    axes[i].set_xlabel("# demonstrations")
    axes[i].set_title(unique_name)
    i += 1
    
axes[0].set_ylabel("Task success metric")

fig = matplotlib.pyplot.gcf()
fig.set_size_inches(15, 4)
fig.savefig("MTBSE2violins.pdf", bbox_inches="tight")
plt.show()

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def create_boxplot_plot_for_name_on_axes(dfs, name, ax_on):
    df_all = pd.concat(dfs)
    df_all.head()
    print name
    df_for = df_all[df_all['name'] == name]
    df_for = df_for[df_for['reward'] > -40]
    
    sns.set(style="whitegrid")
    sns.boxplot(x=df_for["num_logs"], y=df_for["reward"],  order=["30", "50", "100", "200"], ax=ax_on, whis=1e9)
    #sns.swarmplot(x=df_for["num_logs"], y=df_for["reward"],  order=["30", "50", "100", "200"], ax=ax_on)
        
f, axes = plt.subplots(1, 4, figsize=(7, 3), sharex=True)
    
for i, unique_name in enumerate(['gt-3D-points', 'gt-3D-points-projected', 'dd-d_images-SURF-16-10D', 'endtoend-fullwidth']):
    create_boxplot_plot_for_name_on_axes(dfs, unique_name, axes[i])
    axes[i].set(ylim=(-10,1))
    if i > 0:
        axes[i].set_ylabel("")
    axes[i].set_xlabel("# logs")
    axes[i].set_title(unique_name)
    

fig = matplotlib.pyplot.gcf()
fig.set_size_inches(15, 4)
plt.show()

In [ ]:
np.inf